In [1]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.2 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import faiss

In [9]:
faiss.__version__

'1.13.2'

In [60]:
bio_text = """
This lecture focuses on practical implementation of vector databases, specifically using FAISS (Facebook AI Similarity Search). The session begins with a review of vector databases, their advantages over SQL and NoSQL databases, and their application in Retrieval Augmented Generation (RAG). The core of the lecture involves setting up and using FAISS with Python, including installation, data preparation, embedding generation, and similarity search.

Key topics covered:

FAISS Introduction: Overview of FAISS as a vector database and its full form (Facebook AI Similarity Search).
Setup and Installation: Step-by-step guide to installing FAISS using pip install faiss-cpu.
Data Preparation:
Importance of converting unstructured text data into numerical embeddings.
Discussion on chunking text data into smaller, manageable pieces for better search results.
Explanation of chunk size and overlap to maintain context between chunks.
Embedding Generation:
Using the Euron API to generate embeddings for text chunks.
Explanation of API usage, including API URL, API key, and model selection (text embedding-3-small).
Converting embeddings into a float32 matrix format using NumPy for FAISS compatibility.
FAISS Implementation:
Creating an index in FAISS to store embeddings.
Explanation of the index_flat_l2 function and its use of Euclidean distance for similarity search.
Adding embeddings to the FAISS index.
Persisting the FAISS index to a file using faiss.write_index.
Similarity Search:
Performing similarity searches using a query text.
Converting the query text into an embedding.
Using the FAISS index to find the most similar embeddings to the query.
Explanation of the search function and the top_k parameter.
Practical Demonstration:
The lecture includes a live coding session where the instructor demonstrates the entire process from installation to similarity search.
The instructor addresses common issues and errors encountered during the implementation.
Key Takeaways:
Vector databases are essential for similarity search and RAG applications.
FAISS is a powerful tool for building vector search systems.
Proper data preparation and embedding generation are crucial for accurate search results.
Chunking and overlap can significantly impact search performance.
The Euron API provides a convenient way to generate embeddings.
Main learning objectives include understanding the practical steps involved in setting up and using FAISS for similarity search, as well as grasping the importance of data preparation and embedding generation in vector database applications.
"""

In [61]:
import requests
import os
import json
import numpy as np
import requests
import json

In [63]:
def chunk_text(text, chunk_size = 50, overlap= 5):
    words = text.split()
    chunks =[]
    start = 0
    while start < len(words):
        end = min(start+chunk_size, len(words))
        chunk  = ' '.join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

In [64]:
chunks  = chunk_text(bio_text)

In [65]:
chunks

['This lecture focuses on practical implementation of vector databases, specifically using FAISS (Facebook AI Similarity Search). The session begins with a review of vector databases, their advantages over SQL and NoSQL databases, and their application in Retrieval Augmented Generation (RAG). The core of the lecture involves setting up and using',
 'involves setting up and using FAISS with Python, including installation, data preparation, embedding generation, and similarity search. Key topics covered: FAISS Introduction: Overview of FAISS as a vector database and its full form (Facebook AI Similarity Search). Setup and Installation: Step-by-step guide to installing FAISS using pip install faiss-cpu. Data',
 'using pip install faiss-cpu. Data Preparation: Importance of converting unstructured text data into numerical embeddings. Discussion on chunking text data into smaller, manageable pieces for better search results. Explanation of chunk size and overlap to maintain context between c

Embeddings for the above created chunks with text-embedding-3-small model

In [66]:
API_URL = "https://api.euron.one/api/v1/euri/embeddings"
API_KEY = "euri-15626beca1faa283e4147b25cfae0696f11e76f12cc85a0aab76aaa9e91b8839"
MODEL_NAME = "text-embedding-3-small"

In [67]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

In [68]:
all_embeddings = []
for i, chunk in enumerate(chunks):
    payload = {
        "model": MODEL_NAME,
        "input": chunk
    }
    
    response = requests.post(API_URL, headers=headers, data=json.dumps(payload))

    data = response.json()
    embeddings = data['data'][0]['embedding']
    all_embeddings.append(embeddings)

In [69]:
len(all_embeddings)

9

In [70]:
len(chunks)

9

In [23]:
result['data'][0]['embedding']

[-0.024539273,
 -0.018892333,
 0.034407575,
 0.013632929,
 0.020982254,
 -0.013224633,
 0.033411056,
 0.09610869,
 0.031639468,
 -0.0037127242,
 0.038781185,
 -0.06028938,
 -0.082268156,
 -0.016138066,
 0.02732122,
 0.020746965,
 -0.024566954,
 -0.06272531,
 -0.016207268,
 0.04459421,
 0.047888257,
 0.04171538,
 0.036483657,
 0.009051711,
 0.022587756,
 -0.07119572,
 -0.0026435426,
 0.02214486,
 -0.03299584,
 -0.037258726,
 0.035874672,
 -0.027957885,
 -0.018670885,
 0.027196655,
 -0.023556594,
 0.006487752,
 -0.012989344,
 -0.008511931,
 -0.005847627,
 0.021494355,
 -0.015003142,
 -0.033992358,
 -0.005854547,
 0.014878578,
 -0.035930034,
 -0.020774646,
 -0.03914104,
 0.003171213,
 -0.006366647,
 0.055500552,
 -0.025701877,
 0.018864652,
 0.007397767,
 -0.030476863,
 -0.045784496,
 0.006857986,
 0.029535707,
 0.03969466,
 -0.050102744,
 -0.012145071,
 0.024912966,
 -0.021369789,
 0.097215936,
 0.012657171,
 0.011840579,
 -0.008498089,
 -0.04667029,
 0.0074877306,
 -0.035680905,
 -0.020

now store the embeddded data to FAISS vector db

1. to do that need to convert the data to float 32 

In [72]:
embeddings_array = np.array(all_embeddings, dtype='float32')


in any vector db by the time of loading data in it before that the type of similarity search has to be defined either it should be :
1. eucledian 
2. cosine etc.

In [73]:
base_index = faiss.IndexFlatL2(embeddings_array.shape[1]) # Euclidean

In [74]:
base_index.add(embeddings_array)

In [ ]:
faiss.write_index(base_index, "faiss_index.faiss") # writing index as a serializable file.

Search Testing

In [106]:
query_test = "what is ndex_flat_l2"

In [77]:
def embedding_text(text):
    payload = {
        "model": MODEL_NAME,
        "input": chunk
    }
    
    response = requests.post(API_URL, headers=headers, data=json.dumps(payload))

    data = response.json()
    embeddings = data['data'][0]['embedding']
    emb = np.array(embeddings, dtype='float32').reshape(1,-1)
    return emb

In [107]:
query_text_emb = embedding_text(query_test)

In [109]:
base_index.search(query_text_emb,3)

(array([[0.       , 1.2713995, 1.288696 ]], dtype=float32), array([[8, 0, 6]]))

In [108]:
base_index.search(query_text_emb,3)

(array([[0.       , 1.2713995, 1.288696 ]], dtype=float32), array([[8, 0, 6]]))

In [98]:
chunks[6]

'and errors encountered during the implementation. Key Takeaways: Vector databases are essential for similarity search and RAG applications. FAISS is a powerful tool for building vector search systems. Proper data preparation and embedding generation are crucial for accurate search results. Chunking and overlap can significantly impact search performance. The Euron'